<a href="https://colab.research.google.com/github/lalanikarim/notebooks/blob/main/Intro_to_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spoil-A-Wish with LangChain and Gradio

Spoil-A-Wish is a fun little game you play with friends where you make a wish and your friend grants it with a twist, essentially ruining it.

For example:

```
Wish: I wish for a million dollars.  
Response: Granted, but you get a million dollars in ZWL (Zimbabwe's defunct currency) which equals $15 USD. Enjoy your happy meal.
```

In this sample project, we will create a simple scaffolding that can be used to build an AI Agent that spoils your wishes.

## 1. Install python dependencies

In [1]:
!pip install -q langchain-core

## 2. Import packages

In [2]:
from langchain_core.prompts import PromptTemplate

## 3. Prepare Prompt Template

In [3]:
prompt_template = """
Consider your wish, "{wish}", spoiled!!!
"""

prompt = PromptTemplate.from_template(prompt_template)

### Test Prompt Template

In [4]:
wish = "I wish for a million dollars"
response = prompt.invoke({"wish": wish})
response.text.strip()

'Consider your wish, "I wish for a million dollars", spoiled!!!'

## 4. Build a UI with Gradio

Gradio is an open-source Python package that allows you to quickly build a demo or web application for your machine learning model, API, or any arbitrary Python function. You can then share a link to your demo or web application in just a few seconds using Gradio's built-in sharing features. No JavaScript, CSS, or web hosting experience needed! -https://gradio.app

### Install Gradio package

In [5]:
!pip install -q gradio

### Import Gradio module

In [6]:
import gradio as gr

### Function callback

In [7]:
def spoil_wish(wish):
    response = prompt.invoke({"wish": wish})
    return response.text.strip()

### Simple Gradio [Interface](https://www.gradio.app/docs/gradio/interface)  

Interface offers a simple 2 column layout with inputs on the left and outputs on the right.

Interface takes the following parameters:

1. fn: callback function takes inputs and returns outputs
2. inputs: component or a list of components to capture inputs
3. outputs: component or a list of components to send the outputs of the callback function to

In [ ]:
app = gr.Interface(
    fn=spoil_wish,
    inputs="text",
    outputs="text"
)

app.launch()

### Gradio [Components](https://www.gradio.app/docs/gradio/introduction)

Gradio provides a bunch of components to handle various types of inputs and outputs from simple text fields, to bar plots, to audio and video controls, to name a few.

In [ ]:
app = gr.Interface(
    fn=spoil_wish,
    inputs=gr.Textbox(label="Wish", value="I wish for a million dollars"),
    outputs=gr.Textbox(label="Response", lines=3)
)

app.launch()

### Gradio [Blocks](https://www.gradio.app/docs/gradio/blocks)

Gradio allows you to create custom layouts using Blocks.  
You can trigger the callback function to be called on button click event.

In [ ]:
with gr.Blocks(css="""
    .gradio-container {
        display: flex;
        flex-direction: column;
        max-width: 1000px !important;
    }
    .btn.success {
        background-color: #4CAF50;
        color: white;
    }
    .btn.danger {
        background-color: #F44336;
        color: white;
    }
""") as app:
    gr.HTML("""
    <center>
      <h1>Spoil-A-Wish</h1>
    </center>
    """)
    with gr.Row():
        with gr.Column():
            wish = gr.TextArea(label="Wish", value="I wish for a million dollars")
            response = gr.TextArea(label="Response")
            submit = gr.Button("Spoil Wish", elem_classes="btn success")

    submit.click(
        fn=spoil_wish,
        inputs=wish,
        outputs=response
    )

app.launch()

### Designing complex pipelines

Gradio allows you to chain and stack events to create complex UI/UX pipelines.

In [ ]:
with gr.Blocks(css="""
    .gradio-container {
        display: flex;
        flex-direction: column;
        max-width: 1000px !important;
    }
    .btn.success {
        background-color: #4CAF50;
        color: white;
    }
    .btn.danger {
        background-color: #F44336;
        color: white;
    }
""") as app:
    gr.HTML("""
    <center>
      <h1>Spoil-A-Wish</h1>
    </center>
    """)
    with gr.Row():
        with gr.Column():
            wish = gr.TextArea(label="Wish", value="I wish for a million dollars")
            response = gr.TextArea(label="Response", visible=False)
            submit = gr.Button("Spoil Wish", elem_classes="btn success")
            reset = gr.Button("Try Again!", elem_classes="btn danger", visible=False)

    gr.on(
        triggers=submit.click,
        outputs=[wish, response, submit],
        fn=lambda: [gr.TextArea(interactive=False), gr.TextArea(visible=True), gr.Button(visible=False)],
        api_name=False
    ).then(
        fn=spoil_wish,
        inputs=wish,
        outputs=response
    ).then(
        outputs=reset,
        fn=lambda: gr.Button(visible=True),
        api_name=False
    )
    gr.on(
        triggers=reset.click,
        outputs=[response, reset, wish, submit],
        fn=lambda: [gr.TextArea(visible=False), gr.Button(visible=False), gr.TextArea(interactive=True, value=None), gr.Button(visible=True)],
        api_name=False
    )

app.launch()

### Gradio [Examples](https://www.gradio.app/docs/gradio/examples) helper

Gradio Examples lets you specift a dataset of curated inputs which can be tied to input components. This can come in handy when designing applications with multiple input fields and you want to provide preset values for those fields.

In [ ]:
with gr.Blocks(css="""
    .gradio-container {
        display: flex;
        flex-direction: column;
        max-width: 1000px !important;
    }
    .btn.success {
        background-color: #4CAF50;
        color: white;
    }
    .btn.danger {
        background-color: #F44336;
        color: white;
    }
""") as app:
    gr.HTML("""
    <center>
      <h1>Spoil-A-Wish</h1>
    </center>
    """)
    with gr.Row():
        with gr.Column():
            wish = gr.TextArea(label="Wish")
            response = gr.TextArea(label="Response", visible=False)
            submit = gr.Button("Spoil Wish", elem_classes="btn success")
            reset = gr.Button("Try Again!", elem_classes="btn danger", visible=False)

    gr.Examples(
        examples=[
            ["I wish for a million dollars"],
            ["I wish for a new car"],
            ["I wish for a new house"],
            ["I wish for a new job"]
        ],
        inputs=[wish],
        label="Try these"

    )

    gr.on(
        triggers=submit.click,
        outputs=[wish, response, submit],
        fn=lambda: [gr.TextArea(interactive=False), gr.TextArea(visible=True), gr.Button(visible=False)],
        api_name=False
    ).then(
        fn=spoil_wish,
        inputs=wish,
        outputs=response
    ).then(
        outputs=reset,
        fn=lambda: gr.Button(visible=True),
        api_name=False
    )
    gr.on(
        triggers=reset.click,
        outputs=[response, reset, wish, submit],
        fn=lambda: [gr.TextArea(visible=False), gr.Button(visible=False), gr.TextArea(interactive=True, value=None), gr.Button(visible=True)],
        api_name=False
    )

app.launch()